In [67]:
import random

def roll_d6():
    return random.choice(range(1,7))

def risk_combat(attackers=3, defenders=2, silent=True):
    attackers_0 = attackers
    defenders_0 = defenders
    
    attacker_rolls = sorted([roll_d6() for _ in range(min(attackers, 3))])[::-1]
    defender_rolls = sorted([roll_d6() for _ in range(min(defenders, 3))])[::-1]

    if attacker_rolls[0] <= defender_rolls[0]:
        attackers -= 1
    else:
        defenders -= 1
    try:
        if attacker_rolls[1] <= defender_rolls[1]:
            attackers -= 1
        else:
            defenders -= 1
    except:
        pass
    if not silent:
        print(f'attackers rolled: {attacker_rolls}')
        print(f'defenders rolled: {defender_rolls}')
        print(f'Attackers lose {attackers_0 - attackers}, defenders lose {defenders_0 - defenders}')

    return {
        "attacker_diff": attackers - attackers_0,
        "defender_diff": defenders - defenders_0
    }
    

risk_combat(silent=False)


attackers rolled: [5, 3, 2]
defenders rolled: [5, 2]
Attackers lose 1, defenders lose 1


{'attacker_diff': -1, 'defender_diff': -1}

In [62]:
simulation_results = [risk_combat() for _ in range(1000)]

In [63]:
len(simulation_results)

1000

In [72]:
def risk_war(attacking_army, defending_army, silent=True):
    if attacking_army <= 0:
        return 'Defenders win'
    if defending_army <= 0:
        return 'Attackers win'
    skirmish_results = risk_combat(min(attacking_army, 3), min(defending_army, 2), silent=silent)
    attacking_army += skirmish_results['attacker_diff']
    defending_army += skirmish_results['defender_diff']
    print(f'Attacking army size: {attacking_army}')
    print(f'Defending army size: {defending_army}')
    
    return risk_war(attacking_army, defending_army, silent=silent)

    
risk_war(attacking_army=10, defending_army=10, silent=False)
    

attackers rolled: [4, 4, 3]
defenders rolled: [3, 3]
Attackers lose 0, defenders lose 2
Attacking army size: 10
Defending army size: 8
attackers rolled: [6, 4, 1]
defenders rolled: [3, 1]
Attackers lose 0, defenders lose 2
Attacking army size: 10
Defending army size: 6
attackers rolled: [5, 5, 5]
defenders rolled: [6, 2]
Attackers lose 1, defenders lose 1
Attacking army size: 9
Defending army size: 5
attackers rolled: [5, 5, 4]
defenders rolled: [6, 3]
Attackers lose 1, defenders lose 1
Attacking army size: 8
Defending army size: 4
attackers rolled: [6, 3, 2]
defenders rolled: [6, 4]
Attackers lose 2, defenders lose 0
Attacking army size: 6
Defending army size: 4
attackers rolled: [6, 5, 5]
defenders rolled: [6, 2]
Attackers lose 1, defenders lose 1
Attacking army size: 5
Defending army size: 3
attackers rolled: [4, 2, 1]
defenders rolled: [6, 4]
Attackers lose 2, defenders lose 0
Attacking army size: 3
Defending army size: 3
attackers rolled: [4, 4, 2]
defenders rolled: [5, 2]
Attacke

'Defenders win'